In [1]:
import pandas as pd
import urllib
from sqlalchemy import create_engine
from dateutil.relativedelta import relativedelta

In [2]:
# establish connection to SQL Server

conn_str = (r'Driver={SQL Server};'
                      r'Server=AARSVRSQL;'
                      r'Database=BSCP_DW;'
                      r'Trusted_Connection=yes;')
quoted_conn_str = urllib.parse.quote_plus(conn_str)
conn = create_engine(f'mssql+pyodbc:///?odbc_connect={quoted_conn_str}').connect()

**Fetch Data From DW for Given Claim SID and IRS #**

In [3]:
irs_num = '952367304'
claim_sid = '10196'
query = f'''SELECT FCL.CLAIM_SID, FCL.MEMBER_SID, FCL.LINE_BILLED_AMT, 
    FCL.DISCHARGE_DT, FCL.ADMIT_DT, DCH.PAT_STATUS_CD, MBR.MBR_BIRTH_DT, FCL.LINE_FROM_DT,
    FCL.OTHER_PAYER_PAID_AMT, FCL.LINE_COPAY_AMT, DCH.DRG_CD_PAID, RND.PROV_IRS_NUM AS RND_PROV_IRS_NUM,
    REF.PROV_IRS_NUM AS REF_PROV_IS_NUM, BIL.PROV_IRS_NUM AS BIL_PROV_IRS_NUM, PCP.PROV_IRS_NUM AS PCP_PROV_IRS_NUM

    FROM PHI.DIM_CLAIM_HEADER AS DCH --CLAIM HEADER LEVEL DATA
    INNER JOIN PHI.FACT_CLAIM_LINE AS FCL --CLAIM LINE LEVEL MEASURES
        ON FCL.CLAIM_SID = DCH.CLAIM_SID
    INNER JOIN PHI.DIM_MEMBER AS MBR --MEMBER JOIN
        ON FCL.MEMBER_SID = MBR.MEMBER_SID
    INNER JOIN PHI.DIM_PROVIDER AS RND --RENDERING PROVIDER JOIN
        ON FCL.PROV_RENDERING_SID = RND.PROVIDER_SID
    INNER JOIN PHI.DIM_PROVIDER AS REF --REFERRING PROVIDER JOIN
        ON FCL.PROV_REFERRING_SID = REF.PROVIDER_SID
    INNER JOIN PHI.DIM_PROVIDER AS BIL --BILLING PROVIDER JOIN
        ON FCL.PROV_BILLING_SID = BIL.PROVIDER_SID
    INNER JOIN PHI.DIM_PROVIDER AS PCP --PCP PROVIDER JOIN
        ON FCL.PROV_PCP_SID = PCP.PROVIDER_SID
    WHERE BIL.PROV_IRS_NUM = {irs_num}
    AND FCL.CLAIM_SID = {claim_sid}'''


In [4]:
data = pd.read_sql(query, conn)
data.head()

,CLAIM_SID,MEMBER_SID,LINE_BILLED_AMT,DISCHARGE_DT,ADMIT_DT,PAT_STATUS_CD,MBR_BIRTH_DT,LINE_FROM_DT,OTHER_PAYER_PAID_AMT,LINE_COPAY_AMT,DRG_CD_PAID,RND_PROV_IRS_NUM,REF_PROV_IS_NUM,BIL_PROV_IRS_NUM,PCP_PROV_IRS_NUM
0,10196,99394,15450.6,None,2018-11-01,30,1922-06-16,2018-11-01,0.0,0.0,,952367304,None,952367304,None


**Fetch DRG Table from Calculator Requirements Spreadsheet**

In [5]:
drg_table = pd.read_excel('T:\AArete Team Member Folders\msimpson\Promise\DRG_Calc_Requirements_20211006v2.xlsx', sheet_name='DRG Table')
drg_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1306 entries, 0 to 1305
Data columns (total 10 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   APR-DRG                             1306 non-null   object 
 1   APR-DRG Description                 1306 non-null   object 
 2   National Average Length of Stay     1306 non-null   float64
 3   HSRV Casemix Relative Weight        1306 non-null   float64
 4   Policy Adjustor A                   1306 non-null   object 
 5   Policy Adjustor B                   1306 non-null   object 
 6   Policy Adjustor C                   1306 non-null   object 
 7   Policy Adjustor D                   1304 non-null   float64
 8   Medicaid Care Category-- Adult      1304 non-null   object 
 9   Medicaid Care Category-- Pediatric  1304 non-null   object 
dtypes: float64(3), object(7)
memory usage: 102.2+ KB


**Fetch Hospital Characteristics Table from Calculator Requirements Spreadsheet**

In [6]:
hosp_char_table = pd.read_excel('T:\AArete Team Member Folders\msimpson\Promise\DRG_Calc_Requirements_20211006v2.xlsx', sheet_name='Hospital Characteristics')
hosp_char_table = hosp_char_table[hosp_char_table['OSHPD ID'] == 106370875]
hosp_char_table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1 entries, 330 to 330
Data columns (total 15 columns):
 #   Column                                                         Non-Null Count  Dtype  
---  ------                                                         --------------  -----  
 0   OSHPD ID                                                       1 non-null      float64
 1   Provider Name                                                  1 non-null      object 
 2   DPH                                                            1 non-null      object 
 3   NDPH                                                           1 non-null      object 
 4   Designated NICU as Defined by DHCS                             1 non-null      object 
 5   OSHPD Rural Hospital                                           1 non-null      object 
 6   DHCS Designated Remote Rural                                   1 non-null      object 
 7   SFY 2020-21 Cost-to-Charge Ratio                              

In [11]:
'''
Function to get components of DRG price calculator and final payout.

:param df: pd.DataFrame from DW for claim
:param drg_table: pd.DataFrame containing DRG table
:param hosp_char_table: pd.DataFrame containing hospital characteristics table

:return components: dict containing components of calculator
:return final_payment: float final payout after applying calculator
'''
def get_components(df, drg_table=drg_table, hosp_char_table=hosp_char_table):
    comp_names = ['CLAIM_SID', 'Total Charges', 'Cost:Charge', 'Length of Stay', 'Patient Discharge ST == Transfer', 
                  'Patient Age', 'Other Health Coverage', 'Patient Share of Cost', 'Discharge == 30', 'NICU Facility', 
                 'DRG Code', 'DRG Code Description', 'Casemix Weight', 'Policy Adj. Name', 'Policy Adj. Value', 'Payment Relative Weight', 
                 'Avg. Length of Stay for DRG', 'DRG Base Rate', 'Cost Outlier Thresh', 'Marginal Cost %', 'Casemix Adj.', 'Interim Claim Thresh', 
                 'Interim Per Diem Amnt', 'Discharge Status == 30', 'Length of Stay > Interim Thresh', 'Final Interim Claim Payment Amnt', 
                 'DRG Base Payment', 'Transfer Adj. Applicable', 'Transfer Payment Adj. Amnt', 'Transfer Payment Adj. < Allowed Amnt', 'Allowed Amnt After Transfer Adj.', 
                  'Estimated Cost', 'Estimated Cost > Allowed Amnt', 'Estimated Loss', 'Loss > Outlier Thresh', 'DRG Cost Outlier Payment Increase', 
                  'Estimated Gain', 'Gain > Outlier Thresh', 'DRG Cost Outlier Payment Decrease', 'DRG Payment', 'Allowed Amnt', 'Patient Share of Cost', '"Lesser Of" Calc',
                 ]
    components = {name: None for name in comp_names}
    components['CLAIM_SID'] = df['CLAIM_SID'].unique()[0]
    components['Total Charges'] = df['LINE_BILLED_AMT'].sum()
    components['Cost:Charge'] = hosp_char_table["SFY 2020-21 Cost-to-Charge Ratio"].iloc[0]
    
    components['Length of Stay'] = (pd.to_datetime(df['DISCHARGE_DT']) - pd.to_datetime(df['ADMIT_DT'])).value_counts(ascending=False, dropna=False).index[0]
    components['Patient Discharge ST == Transfer'] = int(df['PAT_STATUS_CD'].isin(['02', '05', '63', '65', '66', '82',  '85', '91', '93', '94']).any())
    components['Patient Age'] = relativedelta(pd.to_datetime(df['LINE_FROM_DT'].value_counts(ascending=False, dropna=False).index[0]), pd.to_datetime(df['MBR_BIRTH_DT']).value_counts(ascending=False, dropna=False).index[0]).years
    components['Other Health Coverage'] = df['OTHER_PAYER_PAID_AMT'].sum()
    components['Patient Share of Cost'] = df['LINE_COPAY_AMT'].sum() 
    components['Discharge == 30'] = int((df['PAT_STATUS_CD'] == 30).any())
    
    if (hosp_char_table['Designated NICU as Defined by DHCS'] == 'Yes').any():
        components['NICU Facility'] = True 
    elif (hosp_char_table['Designated NICU as Defined by DHCS'] == 'No').any():
        components['NICU Facility'] = False
    else:
        components['NICU Facility'] = float('nan')
        
    components['DRG Code'] = df['DRG_CD_PAID'].value_counts(ascending=False, dropna=False).index[0]
    
    drg_info = drg_table.loc[drg_table['APR-DRG'].str.contains('380')].iloc[0]
    components['DRG Code Description'] = drg_info['APR-DRG Description'].strip()
    components['Casemix Weight'] = drg_info['HSRV Casemix Relative Weight']
    
    if components['Patient Age'] >= 21 and components['NICU Facility'] == 'No':
        components['Policy Adj. Name'] = 'A'
    elif components['Patient Age'] >= 21 and components['NICU Facility'] == 'Yes':
        components['Policy Adj. Name'] = 'B'
    elif components['Patient Age'] < 21 and components['NICU Facility'] == 'No':
        components['Policy Adj. Name'] = 'C'
    elif components['Patient Age'] < 21 and components['NICU Facility'] == 'Yes':
        components['Policy Adj. Name'] = 'D'
    
    try:
        components['Policy Adj. Value'] = drg_info[f"Policy Adjustor {components['Policy Adj. Name']}"]
    except KeyError: # if Policy Adj. Name is None, there will be no value
        components['Policy Adj. Value'] = float('nan')
    
    components['Payment Relative Weight'] = components['Casemix Weight'] * components['Policy Adj. Value']
    components['Avg. Length of Stay for DRG'] = drg_info['National Average Length of Stay']
    components['DRG Base Rate'] = hosp_char_table['SFY 2020-21 Wage Adjusted Base Rate'].iloc[0]
    
#     if components['Total Charges'] > 61000:
#         components['Cost Outlier Thresh'] = True
#     else:
#         components['Cost Outlier Thres'] = False
    components['Cost Outlier Thresh'] = 61000
        
    components['Marginal Cost %'] = 55
    components['Casemix Adj.'] = 1
    components['Interim Claim Thresh'] = 29
    components['Interim Per Diem Amnt'] = 600
    
    if int(df['PAT_STATUS_CD'].value_counts(ascending=False, dropna=False).index[0]) == 30:
        components['Discharge Status == 30'] = True
    else:
        components['Discharge Status == 30'] = False
    
    if components['Discharge Status == 30']:
        if components['Length of Stay'].days > components['Interim Claim Thresh']:
            components['Length of Stay > Interim Thresh'] = True
        else:
            components['Length of Stay > Interim Thresh'] = False
    else:
        components['Length of Stay > Interim Thresh'] = float('nan')
    
    if components['Length of Stay > Interim Thresh']:
        components['Final Interim Claim Payment Amnt'] = components['Length of Stay'].days * components['Interim Per Diem Amnt']
    elif not components['Length of Stay > Interim Thresh']:
        components['Final Interim Claim Payment Amnt'] = 0
    else:
        components['Final Interim Claim Payment Amnt'] = float('nan')

    components['DRG Base Payment'] = components['DRG Base Rate'] * components['Payment Relative Weight'] * components['Casemix Adj.']
    components['Transfer Adj. Applicable'] = components['Discharge Status == 30']
    
    if components['Transfer Adj. Applicable']:
        components['Transfer Payment Adj. Amnt'] = components['DRG Base Payment'] / components['Avg. Length of Stay for DRG'] * (components["Length of Stay"] + 1)
    else:
        components['Transfer Adj. Applicable'] = float('nan')
    
    if components['Transfer Adj. Applicable'] == float('nan'):
        components['Transfer Payment Adj. < Allowed Amnt'] = float('nan')
    elif components['Transfer Adj. Applicable'] < components['DRG Base Payment']:
        components['Transfer Payment Adj. < Allowed Amnt'] = True
    else:
        components['Transfer Adj. Applicable'] = False
    
    if components['Transfer Adj. Applicable']:
        components['Allowed Amnt After Transfer Adj.'] = components['Transfer Payment Adj. Amnt']
    else:
        components['Allowed Amnt After Transfer Adj.'] = components['DRG Base Payment']
    
    components['Estimated Cost'] = components['Total Charges'] * components['Cost:Charge']
    
    if components['Estimated Cost'] > components['Allowed Amnt After Transfer Adj.']:
        components['Estimated Cost > Allowed Amnt'] = True # loss
    else:
        components['Estimated Cost > Allowed Amnt'] = False # gain
    
    if components['Estimated Cost > Allowed Amnt']: # loss
        components['Estimated Loss'] = components['Estimated Cost'] - components['Allowed Amnt After Transfer Adj.']
    else:
        components['Estimated Loss'] = float('nan')
    
    if components['Estimated Cost > Allowed Amnt']: # loss
        if components['Estimated Loss'] > components['Outlier Thres']:
            components['Loss > Outlier Thresh'] = True
        else:
            components['Loss > Outlier Thresh'] = False
    else:
        components['Loss > Outlier Thresh'] = False
    
    if ['Loss > Outlier Thresh']:  
        components['DRG Cost Outlier Payment Increase'] = (components['Estimated Loss'] - components['Cost Outlier Thresh']) * components['Marginal Cost %'] / 100
    else:
        components['DRG Cost Outlier Payment Increase'] = 0
    
    if not components['Estimated Cost > Allowed Amnt']: # gain
        components['Estimated Gain'] = components['Allowed Amnt After Transfer Adj.'] - components['Estimated Cost']
    else:
        components['Estimated Gain'] = float('nan')
        
    if not components['Estimated Cost > Allowed Amnt']: # gain
        if components['DRG Cost Outlier Payment Increase'] > components['DRG Base Rate']:
            components['Gain > Outlier Thresh'] = True
        else:
            components['Gain > Outlier Thresh'] = False
    else:
        components['Gain > Outlier Thresh'] = float('nan')
    
    if not components['Estimated Cost > Allowed Amnt'] and components['Gain > Outlier Thresh']:
        components['DRG Cost Outlier Payment Decrease'] = components['Marginal Cost %'] / 100 * (components['Estimated Gain'] - components['Cost Outlier Thresh'])
    else:
        components['DRG Cost Outlier Payment Decrease'] = 0
    
    if components['Estimated Cost > Allowed Amnt']:
        components['DRG Payment'] = components['Allowed Amnt After Transfer Adj.'] + components['DRG Cost Outlier Payment Increase']
    else:
        components['DRG Payment'] = components['Allowed Amnt After Transfer Adj.'] - components['DRG Cost Outlier Payment Decrease']
        
    components['Allowed Amnt'] = components['DRG Payment']
    components['"Lesser Of" Calc'] = min(components['Total Charges'], components['Allowed Amnt'])
    
    if components['Discharge == 30']:
        final_payment = components['Final Interim Claim Payment Amnt']
    else:
        final_payment = components['"Lesser Of" Calc'] - components['Final Interim Claim Payment Amnt'] - components['Other Health Coverage'] - components['Patient Share of Cost']
    
    return components, final_payment

In [12]:
comp, pay = get_components(data)

<ipython-input-11-d198fac789f5>:95: RuntimeWarning: invalid value encountered in greater
  elif components['Transfer Adj. Applicable'] < components['DRG Base Payment']:
<ipython-input-11-d198fac789f5>:136: RuntimeWarning: invalid value encountered in less
  if components['DRG Cost Outlier Payment Increase'] > components['DRG Base Rate']:
